In [117]:
class Layer:
    def __init__(self, n_neurons, act_func: str = 'sigmoid'):
        self.n_neurons = n_neurons
        self.act_func = act_func
        self.units = None
    def activation(self, w,X):
        if self.act_func == 'relu':
            return np.maximum(0, X@w)
        if self.act_func == 'sigmoid':
            return 1/(1+np.exp(-X@w))
    def call(self, X, w):
        return self.activation(w, X)

In [3]:
class NeuralNetwork:
    def __init__(self, n_layers, n_epox = 5, learning_rate = 0.1):
        self.n_layers = n_layers
        self.n_epox = n_epox
        self.learning_rate = learning_rate
        self.num_neurons = [7, 64, 128, 1]
    def sigmoid(self, x, grad = False):
        s = 1/(1+np.exp(-x))
        if grad == True:
            return s@(s-1)
        else:
            return s
        
    def fit(self, X, y):
        self.W = []
        for i in range(1, self.n_epox+1):
            print(i)
            if i ==1:
                for j in range(1, self.n_layers+1):
                    if j == 1:
                        w = np.random.normal(0, 1/64, (X.shape[1], self.num_neurons[j-1]))
                        self.W.append(w)
                    else:
                        w = np.random.normal(0, 1/self.num_neurons[j-1], (self.num_neurons[j-2], self.num_neurons[j-1]))
                        self.W.append(w)
                output = self.forward_prop(X, self.W)
                #print(output[-1])
                self.W = self.back_prop(X, y, output, self.W)
            else:
                output = self.forward_prop(X, self.W)
                self.W = self.back_prop(X, y, output, self.W)
            print(np.sum((self.outputs[-1]-y)**2, axis = 0)[0])#for print loss
    def back_prop(self, X, y, output, W):
        gradients = []
        n_layers = len(W)
        y = np.array(y)
        y = np.reshape(y, (len(y), 1))
        print(output[-1].shape)
        # Calculate the gradient of the loss with respect to the output
        gradient_loss = output[-1] - y
        print(gradient_loss.shape)
        # Backpropagate the gradients through each layer
        gradient_back = gradient_loss
        for i in range(n_layers, 0, -1):
            gradient_layer = gradient_back*output[i-1]#element wise
            gradient_back = np.dot(W[i-1], gradient_layer.T).T# * output[i-1] * (1 - output[i-1])
            gradients.append(gradient_back)

        # Update the weights using the gradients
        for i in range(n_layers):
            W[i] -= self.learning_rate * gradients[i-1]

        return W
            
    def forward_prop(self, X, w): 
        self.outputs = []#save outputs
        for i in range(1, self.n_layers+1):
            if i == 1:
                layer = Layer(self.num_neurons[i], 'sigmoid')

                self.outputs.append(layer.call(X, w[i-1]))
            else:
                self.outputs.append(layer.call(self.outputs[i-2], w[i-1]))
        return self.outputs
    
  
    
        
        
        
            

In [2]:
import pandas as pd
import numpy as  np
from sklearn.model_selection import train_test_split
data = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])
X = data.drop('Age', axis = 1)
y = data['Age']
x_train, x_test, y_train, y_test = train_test_split(X, y,random_state=0, test_size=0.2)


In [192]:
model = NeuralNetwork(3)
model.fit(x_train, y_train)

1
(7, 64)
5


ValueError: Dot product shape mismatch, (2656, 64) vs (7, 64)